### Meraki Rate Limiting Exercise

This notebook illustrates how to implement logic to handle failures due to rate limits on the Meraki Cloud.
For an example code, refer to [*tips to avoid being rate limited*](https://developer.cisco.com/meraki/api/#/rest/guides/rate-limit/tips-to-avoid-being-rate-limited)

In this demonstration we are using the requests module rather than the Meraki SDK. This lab provides a perspective of the benefits of using a SDK rather than managing the APIs directly from your code. 

** Imports **

In addition to importing `json` and `requests`, the `time` module is used to sleep (suspend execution) in the rate limit function.

In [1]:
import time
import json
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()

**Constants**

Define constants to control the program execution. (Feel free to modify these if you wish). Because this lab is attempting to trigger the rate limit logic, the program must issue API calls in a rapid succession to trigger the logic. The goal is to trigger a 429 response code to test our logic.

The nature of the rate limit logic is to recover from the error and be successful on a subsequent execution of the API call. However, there must be some limit or bounds on how many attempt before we give up! 

In [2]:
ITERATIONS = 10                         # Number of API calls to issue to attempt a trigger of the rate limit response
RL_RETRY = 4                            # Number of attempts to issue the command before assuming the 429 is persistent

**Friendly Names**

Rather than use the integers 429 and 200 in our program, give them human friendly names so the code is more readable. `OK` is defined as a tuple with one value. The reason? There might be other return codes which are acceptable in addition to 200, so create a tuple which can be referenced in the code.

In [3]:
RATE_LIMIT_EXCEEDED = 429
OK = (200,)

**Headers and URL(s)**

One advantage of the SDK, defining the header values is handled by the SDK! Recall from the SDK example in this repository, the API key needed to be specified. The other header information was formatted for us by the SDK.

We will also define a constant `ORG` which holds a URL that can be used to generate API calls to the Meraki dashboard.

The API key used in this example was obtained from: https://developer.cisco.com/meraki/meraki-platform/

In [4]:
header = {"Content-Type": "application/json"}
header["X-Cisco-Meraki-API-Key"] = "6bec40cf957de430a6f1f2baa056b99a4fac9ea0"
ORG = "https://dashboard.meraki.com/api/v0/organizations"

**Logic**

Create a method (a function) to implement the logic to address any rate limit issued we may encounter. In Python, you can assign a function to a variable. This logic provides the ability for our program to call the rate_limit logic from various places in the program. This logic enables the rate limit logic to handle `requests.get`, `requests.put`, `requests.post`, etc.

After executing the function held in `api_call`, check if the `status_code` signals we have been rate limited. Check the header value for a hit from the dashboard as to how log to wait, then try the API call again. At some point we want to exit from this logic, `RL_RETRY` keeps the loop from attempting forever.

In [5]:
def rate_limit(api_call, url, **kwargs):
    
        for _ in range(RL_RETRY):
            response = api_call(url, **kwargs)
            if response.status_code == RATE_LIMIT_EXCEEDED:
                print(f'{RATE_LIMIT_EXCEEDED} {response.headers["X-Request-Id"]}', end=' ')
                time.sleep(int(response.headers.get("Retry-After", 1)))
            else:
                return response
        return response

**Issue a GET**

The method `api_get` provides a means to handle other errors which might be encountered. For example, what happens if we change the URL to a host which doesn't exist? For example:
```
ORG = "https://dashboard.example.net/api/v0/organizations"
```
Try it and see for yourself! Note that when we call the `rate_limit` method, the first argument is the function `requests.get`.

In [6]:
def api_get(uri):
    try:
        r = rate_limit(requests.get, uri, headers=header, verify=False)
    except requests.ConnectionError as e:
        print(f'ConnectionError: {e}')
        return False
    return r


**Test Rate Limiting**

At this point, attemt to trigger the rate limiting logic of the Meraki dashboard. Create a loop, using `ITERATIONS` defined previously, and return the organizations associated with this API key. Triggering the 429 rate limiting errors may require running hundreds of iterations. The function `api_get` returns the requests object, which means we can print out the status code and headers resulting from the request.

In [7]:
for i in range(ITERATIONS):
    r = api_get(ORG)
    if r:
        print(f'{i}|{r.status_code} {r.headers["Date"]} {r.headers["X-Request-Id"]}')
print('Finished!')

0|200 Wed, 02 Jun 2021 14:36:16 GMT 84aedc9418bbfb639b3b0b47aad39035
1|200 Wed, 02 Jun 2021 14:36:18 GMT 108a41a79157e965c3525b27ac6db71f
2|200 Wed, 02 Jun 2021 14:36:19 GMT c046f1d12f194dba922e83a579a37d8f
3|200 Wed, 02 Jun 2021 14:36:20 GMT 072fc4f45cf0bc3fbba0b679c02abbcd
4|200 Wed, 02 Jun 2021 14:36:21 GMT 7c3fc3859eb1e302b1e69bbce8e311b4
5|200 Wed, 02 Jun 2021 14:36:23 GMT a9d7d9118dcd69c24003ffd57513cb44
6|200 Wed, 02 Jun 2021 14:36:24 GMT a83a53ebe8e41902f02af7ce6498c4ef
7|200 Wed, 02 Jun 2021 14:36:25 GMT 3cca49c4ea30721424d6f85f0f4d760e
8|200 Wed, 02 Jun 2021 14:36:26 GMT 6bf410a3bf7ec7261fc91bf22f3deba4
9|200 Wed, 02 Jun 2021 14:36:27 GMT 61c6c82f44b4f362c630a78710c0456b
Finished!


After the loop is finished, you can examine the contents of the *last* returned requests object. The `headers` variable is a dictionary. We referenced some fields previously, now examine all the key, value pairs stored in `headers`.

In [8]:
r.headers

{'Server': 'nginx/1.19.10', 'Date': 'Wed, 02 Jun 2021 14:36:27 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': 'Fri, 01 Jan 1990 00:00:00 GMT', 'X-Frame-Options': 'sameorigin', 'X-Robots-Tag': 'none', 'Sunset': '2022-02-05T23:59:59Z', 'Deprecation': '2020-08-05T23:59:59Z', 'Link': '<https://developer.cisco.com/meraki/api-v1/#!versioning>; rel="deprecation"; type="text/html", <https://n392.meraki.com/api/v1/organizations>; rel="successor-version"; type="application/json"', 'X-UA-Compatible': 'IE=Edge,chrome=1', 'X-Request-Id': '61c6c82f44b4f362c630a78710c0456b', 'X-Runtime': '0.117652', 'X-XSS-Protection': '1; mode=block'}

Similarly, examine the data returned from the API call, in this case, the organizations associated with the provide API key.

In [9]:
r.json()

[{'id': '681155',
  'name': 'DeLab',
  'url': 'https://n392.meraki.com/o/49Gm_c/manage/organization/overview'},
 {'id': '575334852396583259',
  'name': 'Demo Organization',
  'url': 'https://n22.meraki.com/o/OSkPybw/manage/organization/overview'},
 {'id': '575334852396583255',
  'name': 'My organization',
  'url': 'https://n22.meraki.com/o/sZsJbcw/manage/organization/overview'},
 {'id': '575334852396583252',
  'name': 'yanchen0531',
  'url': 'https://n22.meraki.com/o/qDB0zcw/manage/organization/overview'},
 {'id': '575334852396583251',
  'name': 'yanchennewwww',
  'url': 'https://n22.meraki.com/o/7uGP6bw/manage/organization/overview'},
 {'id': '575334852396583247',
  'name': 'FBECK-TEST',
  'url': 'https://n22.meraki.com/o/MU-J1bw/manage/organization/overview'},
 {'id': '575334852396583246',
  'name': 'FBECK-TEST',
  'url': 'https://n22.meraki.com/o/Gpqldaw/manage/organization/overview'},
 {'id': '575334852396583244',
  'name': 'FBECK-TEST',
  'url': 'https://n22.meraki.com/o/UCsuQdw/m

Reference the list of organizations and print out the `id` and `name`.

In [10]:
for organization in r.json():
    print(organization['id'], organization['name'])

681155 DeLab
575334852396583259 Demo Organization
575334852396583255 My organization
575334852396583252 yanchen0531
575334852396583251 yanchennewwww
575334852396583247 FBECK-TEST
575334852396583246 FBECK-TEST
575334852396583244 FBECK-TEST
575334852396583243 Test
575334852396583242 My organization
575334852396583241 My organi
575334852396583240 My organization
575334852396583237 Xorg
575334852396583231 SWTest
575334852396583228 My organization
575334852396583227 My organization
575334852396583223 EvilCorp
575334852396583220 My organization
575334852396583219 BibbyTest
575334852396583218 My organization
575334852396583213 "New Network"
575334852396583212 My organization
575334852396583211 My organization
575334852396583210 My organization
575334852396583209 My organization
575334852396583208 BearHoleComputing
575334852396583207 My organization
575334852396583206 My organization
575334852396583205 My organization
575334852396583204 My organization
575334852396583203 My organization
575334

**Get Networks**

For the next example, our code needs to know what networks are associated with one of the organizations. Issue the API call to get networks for the specified organization, (in this case, 549236), and print out the `id` and network `name`.

In [11]:
r = api_get('https://dashboard.meraki.com/api/v0/organizations/549236/networks')
for network in r.json():
    print(f'{network["id"]} {network["name"]}')


L_646829496481105433 DevNet Sandbox ALWAYS ON
L_646829496481108255 GTECH-MAIN
L_646829496481108285 GTECH
L_646829496481108288 GFORCE
L_646829496481108291 UNPLUGGED SANDBOXES 
L_646829496481108477 ITSCORE
L_646829496481108480 KG
L_646829496481108482 KG_Enterprise
L_646829496481108485 DNSMB2
L_646829496481108486 ABEBI
L_646829496481108492 DNSMB3
L_646829496481108493 DNSMB4
L_646829496481108496 DNENT2


**Issue a POST**

Create a new function to post data to the dashboard. The HTTP POST is commonly used to send some amount of data to the dashboard to update the charactistics of, in this case, a network configuration. This fuction is similar to the `api_get`, with the difference being we have added an additional argument, the `body` variable. The data we intent to send is stored in the `body`.

In [12]:
def api_post(uri, body={}):
    try:
        r = rate_limit(requests.post, uri, data=body, headers=header, verify=False)
    except requests.ConnectionError as e:
        print(f'ConnectionError: {e}')
        return False
    return r

Now invoke the function `api_post`, using one of the network `id` from above in the URL and specify a configuration template in the `body`.

In [13]:
r = api_post(f'https://dashboard.meraki.com/api/v0/networks/{network["id"]}/bind', body='{"configTemplateId": "N_23952905"}')

Examine the result. Why did the POST request fail? Examine the `status_code` and returned `text` field.

In [14]:
print(r.status_code, r.text)

403  


The request failed due to an incorrect template ID. Note the `rate_limit` logic passed the return code *400* back to the calling function. Now, generate another error. This time change the hostname to `example.net`. This host doesn't exist, where will this error be caught in the logic?

In [15]:
r = api_post(f'https://dashboard.example.net/api/v0/networks/{network["id"]}/bind', body='{"configTemplateId": "N_23952905"}')

ConnectionError: HTTPSConnectionPool(host='dashboard.example.net', port=443): Max retries exceeded with url: /api/v0/networks/L_646829496481108496/bind (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fa37c9c2dc0>: Failed to establish a new connection: [Errno -2] Name or service not known'))


The `ConnectionError` message was generated by the try/except block in the `api_post` function. Examine the value of variable `r`.

In [16]:
print(r)

False


Note that the variable `r` has a value of False, a boolean, and `r.status_code` does not exist. The program cannot populate the variable of `r` with a status code because none exists, we were unsuccessful in connecting to the target host, no status code exists. Our program logic returns False in this case. Note that in our looping logic, we check to see `if r` is true before printing the contents of the object.

** Learning Aids **

Study the [List of HTTP Status Codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) paying particular attention to:
```
2xx Success
200 OK
201 Created
202 Accepted

3xx Redirection
301 Moved Permanently

4xx Client errors
400 Bad Request
401 Unauthorized
403 Forbidden
404 Not Found
429 Too Many Requests
```
Many of these common status codes can be observed in this program example. Also familiarize yourself with the definition and usage of the [HTTP request methods](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods).

```
GET requests a representation of the specified resource, retrieves data
POST submit an entity to the specified resource, often causing a change in state or side effects on the server
PUT replaces all current representations of the target resource with the request payload
DELETE deletes the specified resource
PATCH  apply partial modifications to a resource
```

** Summary **

Hopefully this exercise provides a perspective of how a programming must consider and address connection and status error messages when using the Meraki dashboard API directly rather than allowing the SDK to handle these issues. This exercise also introduced the concept of passing functions in variables between different parts of your Python program. It also examines how to incorporate sample logic from the *tips to avoid being rate limited* article and create a more robust solution in your code.

** Author **

Joel W. King (GitHub / GitLab @joelwking)